# Python 3

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import networkx as nx
import scipy.sparse as sp
import pickle
import time
import datetime as dt
from itertools import combinations, chain
from joblib import Parallel, delayed

In [22]:
n_genes = 16271
n_drugs = 639
n_secombo = 964
# Creates random network as graph
start = time.time()
gene_net = nx.planted_partition_graph(200, 80, 0.15, 0.05, seed=42)
ppi_adj = nx.adjacency_matrix(gene_net)
print('PPI adjacency matrix created in ',dt.timedelta(seconds=time.time()-start))
ppi_degrees = np.array(gene_adj.sum(axis=0)).squeeze()

PPI adjacency matrix created in  0:00:29.448602


In [20]:
# Creates random adjacency matrix for genes and drugs
start = time.time()
dti_adj = sp.csr_matrix((10 * np.random.randn(n_genes, n_drugs) > 15).astype(int))
print('DTI adjacency matrix created in ',dt.timedelta(seconds=time.time()-start))

DTI adjacency matrix created in  0:00:00.527054


In [12]:
# DDI adjacency matrices
start = time.time()
tmp = np.dot(drug_gene_adj, gene_drug_adj)
def se_adj_matrix(i):
    mat = np.zeros([n_drugs,n_drugs],dtype=int)
    for d1, d2 in combinations(list(range(n_drugs)), 2):
        if tmp[d1, d2] == i + 5:
            mat[d1, d2] = mat[d2, d1] = 1
    return sp.csr_matrix(mat) 
ddi_adj_list = Parallel(n_jobs=8)\
    (delayed(se_adj_matrix)(d) for d in range(n_secombo))        
ddi_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in ddi_adj_list]
print('DDI adjacency matrices created in ',dt.timedelta(seconds=time.time()-start))

DDI adjacency matrices created in  0:19:19.546477


In [13]:
# Protein Features
prot_feat = sp.identity(n_genes)
# Drug Features
drug_feat = sp.identity(n_drugs)

# Saving

In [14]:
now = dt.datetime.now() # current date and time
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
filename = './data_structures/toyDS_genes'+str(n_genes)+'_drugs'+str(n_drugs)+'_se'\
+str(n_secombo)+'_'+year+'_'+month+'_'+day

./data_structures/toyDS_genes16271_drugs639_se964_2020_05_28


In [23]:
data = {}
# Dictionaries
#data['gene2idx'] = gene2idx
#data['drug2idx'] = drug2idx
#data['se_mono_name2idx'] = se_mono_name2idx
#data['se_combo_name2idx'] = se_combo_name2idx
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat
# PF
data['prot_feat'] = prot_feat
#data['norm_prot_feat'] = norm_prot_feat

In [24]:
with open(filename, 'wb') as f:
    pickle.dump(data, f, protocol=2)